# Release Deployment Control using Gates

Gates allow automatic collection of health signals from external services. These signals are then used to promote the release when all the signals are successful at the same time or stop the deployment on timeout.

In this lab, we will focus on quality metrics from tests such as pass rate and deploy only if they are within required thresholds. Additionally, we will also use a HTTP triggered function in an Azure function app and parse the response.

### New Release Pipeline

1. Create a new release pipeline by selecting `Releases` tab on the left followed by `New pipeline`.

![newReleasePipeline](../images/newReleasePipeline.png)

2. Select `Empty Job` template to add our own steps:

![newReleasePipeline](../images/emptyJobTemplate.png)

3. Ensure there is one stage added under Stages.

4. With `Artifacts`, select _+ Add_. Select `Source type` is `Build`. Provide the `Project` name and `Source (build pipeline)` and add this to artifacts.

5. (Optional) You can also set a continuous deployment trigger by selecting the lightning button in artifacts and then turning on the first enable button. A typical scenario is to create a release every time a new build is available.

### Pre-Deployment and Post-Deployment Approvals ###

Each stage in a release pipeline can be configured with pre-deployment and post-deployment conditions that can include waiting for users to manually approve or reject deployments, and checking with other automated systems until specific conditions are verified. The following diagram shows how these features are combined in a stage of a release pipeline:

![approvalsGates](../images/approvalsGates.png)

By using approvals, gates, and manual intervention you can take full control of your releases to meet a wide range of deployment requirements.

You can define approvals at the start of a stage (pre-deployment approvers), at the end of a stage (post-deployment approvers) or both. The following image illustrates how you can set pre and post deployment conditions:

![prePostIcons](../images/prePostIcons.png)

#### A. Pre-deployment approval ####

For a pre-deployment approval, you may choose the icon at the entry point of the stage and enable pre-deployment approvers. We will not assign any approvers for pre-deployment in this scenario. However, a typical scenario for pre-deployment approvals is when some users must manually validate the change request and approve the deployment to a stage. With gates, you may want to ensure there are no issues in the work item before deploying a build to a stage.

***Triggers***

Triggers will start the deployment and there are two options available: After release and Manual only. In this lab, we will perform manual deployments. To allow only manual deployments, select `Manual`:

![prePostIcons](../images/manualTrigger.png)

***Gates***

We will setup pre-deployment gates to check for published test results and coverage. Follow the below steps to setup a pre-deployment gate:

1. Turn the _Enabled_ button on next to Gates.

2. Click _+ Add_ and select _Invoke Azure Function_

3. Configure the Azure Function Parameters by populating the below fields:

| Name   |      Description      |
|----------|-------------|
| _Azure function URL_ |  The URL of the Azure function to be invoked. Populate this field with the URL of the function we created for monitoring tests. |
| _Function key_ |    The value of the available function or the host key for the function to be invoked. You can set this to $(myFunctionKey)   |
| _Method_ | The HTTP method with which the function will be invoked. Set this field to GET |
| _Headers_ | The header in JSON format to be attached to the request sent to the function. Leave this default. |
| _Query parameters_ | Query parameters to append to the function URL. We will not be passing any query parameters. |
| _Body_ | This is optional and is the request body for the Azure function call in JSON format. |
| _Completion Event_ | This is mandatory and is about how the task reports completion. We will use `ApiResponse` which is default. |
| _Success criteria_ | This is optional and is about how to parse the response body for success. Our function returns `"status: "success"` if all the tests passed. Hence, we will call eq(root['status'], 'success') for validation.|


![azureFunctionConfig](../images/azureFunctionConfig.png)

4. Open the `Evaluation options` tab and change the `The time between re-evaluation of gates` to 5 minutes. This is the duration after which the gates are re-evaluated. This must be greater than the longest typical response time of the configured gates to allow all responses to be received in each evaluation.

#### B. Post-deployment approval #### 

For a post-deployment approval, choose the icon at the exit point of the stage and enable post-deployment approvers. List your username in the Approvers textbox and reduce the timeout to a day. _Note:_ A maximum timeout of 365 days is allowed.

A typical scenario for post-deployment approval is when users must manually sign off the app after deployment before the release is promoted to other stages. With gates, you may want to ensure there are no incidents from the monitoring for the app after it's been deployed, before promoting the release.

#### Create a Release

1. Save the pipeline (by clicking Save at the top right hand corner) by giving a meaningful name to the new release pipeline. This should show up in the releases tab:

![newPipeline](../images/newPipeline.png)

2. Select `Create a Release` to queue a release.

3. In the `Stages` section, select `Deploy` to process the gates. You should see `Processing gates` status in the `Stages` section:

![pendingApproval](../images/processingStages.png)

### Release Logs

An additional feature available with the release tabs is to view the stage logs. Select `Logs` below stage(s) to view the deployment process and gate logs.

![releaseLogs](../images/releaseLogs.png)

This should allow you to view the `Deployment process` where you can see logs from `Azure Function` used in deployment gates and approvals, etc. as shown below:

![deploymentProcess](../images/deploymentProcess.png)

### Approvals

In the final step, the approver would have recieved an email for approval of Deployment to Stage with a link to `View Approval`. On selecting `View Approval`, it will take the approver to the stage to approve as shown below:

![pendingApproval](../images/pendingApproval.png)

If you are the approver, select `Approve` in Stages to provide comments and approve if there are no issues.

Select the `Releases` tab, to see if the release was a succcess. On success, you should see a green tick under stages as shown below:

![successStages](../images/successStages.png)